In [1]:
import torch
import pandas
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import os
import PIL as Image



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Acer nitro\anaconda3\envs\my_new_env\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Acer nitro\anaconda3\envs\my_new_env\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Acer nitro\anaconda3\envs\my_new_env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\Acer nitro\anaconda3\envs\my_new_env\lib\site-packages\traitlets\config\application.py", line 10

### Custom Dataset

In [2]:
class DRIVE_dataset(Dataset):
    def __init__(self,img_dir,mask_dir,transforms = None, mask_transfrom = None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transforms = transforms
        self.mask_transform = mask_transfrom
        self.images = os.listdir(img_dir)
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_name = self.images[index]
        image_path = os.path.join(self.img_dir,image_name)
        mask_path = os.path.join(self.mask_dir,image_name) # considering that masks also have the same name

        image = Image.open(image_path).convert("RGB")        
        mask = Image.open(mask_path)

        if self.transforms:
            image = self.transforms(image)
        if self.mask_transform:
            mask = self.mask_transform(mask)

        return image,mask
        

In [7]:
from torchvision.transforms import ToTensor
train_data = DRIVE_dataset(
    img_dir = "C:/Users/Acer nitro/Desktop/retinal-vessel-segmentation/data/DRIVE/training/images",
    mask_dir = "C:/Users/Acer nitro/Desktop/retinal-vessel-segmentation/data/DRIVE/training/mask",
    transforms = ToTensor(),
    mask_transfrom = None
    )

test_data = DRIVE_dataset(
    img_dir = "C:/Users/Acer nitro/Desktop/retinal-vessel-segmentation/data/DRIVE/test/images",
    mask_dir= "C:/Users/Acer nitro/Desktop/retinal-vessel-segmentation/data/DRIVE/test/mask",
    transforms = ToTensor(),
    mask_transfrom= None
    )

In [8]:
import os
BATCH_SIZE = 4
NO_OF_WORKERS = os.cpu_count()

In [15]:
train_dataloader = DataLoader(train_data,batch_size=BATCH_SIZE,num_workers=NO_OF_WORKERS,shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=BATCH_SIZE,num_workers=NO_OF_WORKERS, shuffle=False)

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [19]:
import torch
from torch import nn

class DoubleConv(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(DoubleConv,self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size=3, padding = 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels,out_channels,kernel_size=3, padding = 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace = True)
            )
    def forward(self,x):
        output = self.double_conv(x)
        return output


class Unet(nn.Module):
    def __init__(self,in_channels = 3,out_channels = 1):
        super(Unet,self).__init__()
        self.enc1 = DoubleConv(in_channels = in_channels ,out_channels =64)
        self.enc2 = DoubleConv(in_channels = 64,out_channels = 128)
        self.enc3 = DoubleConv(in_channels =128,out_channels =256)
        self.enc4 = DoubleConv(in_channels = 256,out_channels = 512)

        self.bottle_neck = DoubleConv(in_channels = 512, out_channels= 1024)

        self.pool = nn.MaxPool2d(kernel_size=2)


      
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec1 = DoubleConv(1024, 512)  # concat with skip connection

        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec2 = DoubleConv(512, 256)  # concat with skip connection

        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec3 = DoubleConv(256, 128)  # concat with skip connection

        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec4 = DoubleConv(128, 64)  # concat with skip connection

        # Final output layer
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)  # For binary segmentation (out_channels = 1)

    def forward(self, x):
        # Encoder
        x1 = self.enc1(x)
        x2 = self.enc2(self.pool(x1))
        x3 = self.enc3(self.pool(x2))
        x4 = self.enc4(self.pool(x3))

        # Bottleneck
        x = self.bottleneck(self.pool(x4))

        # Decoder
        x = self.up1(x)
        x = torch.cat([x, x4], dim=1)
        x = self.dec1(x)

        x = self.up2(x)
        x = torch.cat([x, x3], dim=1)
        x = self.dec2(x)

        x = self.up3(x)
        x = torch.cat([x, x2], dim=1)
        x = self.dec3(x)

        x = self.up4(x)
        x = torch.cat([x, x1], dim=1)
        x = self.dec4(x)

        # Output
        return self.out_conv(x)

In [21]:
model = Unet(in_channels = 3, out_channels = 1).to(device = "cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr = 0.001)

In [23]:
def train_step(model: torch.nn.Module,
               dataloader : torch.utils.data,
               criterion : torch.nn,
               optimizer : torch.optim
        ):
    model.train()
    
    train_loss ,train_dice = 0,0

    for batch ,(X,y) in enumerate(dataloader):
        X,y = X.to(device) , y.to(device)
        
        #1 forward pass
        y_pred = model(X)
        #2 calculate loss
        loss = criterion(y_pred,y)
        train_loss += loss.item()
        #3 gradient zero
        optimizer.zero_grad()
        #4 backprop
        loss.backwards()
        #5 step
        optimizer.step()

        train_dice = dice_score(y_pred,y)
    train_loss /= len(train_dataloader)
    train_dice /= len(train_dataloader)

In [26]:
def test_step(model : torch.nn.Module,
              dataloader :  torch.utils.data,
              criterion : torch.nn):
    model.eval()

    test_loss,test_dice = 0, 0

    for batch,(X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)

        y_pred = model(X)

        loss = criterion(y_pred,y)
        test_loss += loss.item()
        test_dice += dice_score(y_pred,y)

    test_loss /= len(dataloader)
    test_dice /= len(dataloader)


